<a href="https://colab.research.google.com/github/ahmed98Osama/YT-Download-Gofile-Upload/blob/main/Download_YouTube_Videos_Using_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.9 MB/s eta 0:00:00a 0:00:01


In [ ]:

import yt_dlp

def download_youtube_video(video_url, save_path="downloads"):
    """
    Downloads a YouTube video in the best available video and audio quality.

    Args:
        video_url (str): URL of the YouTube video.
        save_path (str): Directory to save the downloaded video.
    """
    try:
        # Options for yt-dlp
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',  # Download best video and audio and merge
            'outtmpl': f'{save_path}/%(title)s.%(ext)s',  # Save file format
            'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
        }

        # Downloading the video
        print("Downloading...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"Video downloaded successfully and saved in: {save_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Input YouTube video URL
    video_url = input("Enter the YouTube video URL: ").strip()

    # Input save directory (default is 'downloads')
    save_path = input("Enter the save path (or press Enter for default 'downloads'): ").strip() or "downloads"

    # Call the function to download the video
    download_youtube_video(video_url, save_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.6 MB/s eta 0:00:00
Enter the YouTube video URL: https://www.youtube.com/watch?v=2wBYW8hnO4A
Enter the save path (or press Enter for default 'downloads'): 
Downloading...
[youtube] Extracting URL: https://www.youtube.com/watch?v=2wBYW8hnO4A
[youtube] 2wBYW8hnO4A: Downloading webpage


[youtube] 2wBYW8hnO4A: Downloading android sdkless player API JSON
[youtube] 2wBYW8hnO4A: Downloading web safari player API JSON


[youtube] 2wBYW8hnO4A: Downloading m3u8 information


[info] 2wBYW8hnO4A: Downloading 1 format(s): 299+251
[download] Destination: downloads/Lost Judgment - All Cutscenes Full Movie (2021).f299.mp4
[download] 100% of   22.95GiB in 00:08:01 at 48.78MiB/s  
[download] Destination: downloads/Lost Judgment - All Cutscenes Full Movie (2021).f251.webm
[download] 100% of  831.96MiB in 00:00:51 at 16.24MiB/s  
[Merger] Merging formats into "downloads/Lost Judgment - All Cutscenes Full Movie (2021).mp4"
Deleting original file downloads/Lost Judgment - All Cutscenes Full Movie (2021).f299.mp4 (pass -k to keep)
Deleting original file downloads/Lost Judgment - All Cutscenes Full Movie (2021).f251.webm (pass -k to keep)
Video downloaded successfully and saved in: downloads


In [ ]:
!curl -F "file=@/content/downloads/DARKSIDERS 3 Gameplay Walkthrough FULL GAME [4K 60FPS PC ULTRA] - No Commentary.mp4" https://upload.gofile.io/uploadfile

{"data":{"createTime":1768649788,"downloadPage":"https://gofile.io/d/NWHuaN","guestToken":"WSrxohH8oPQzbXKYOhBOhYX6JSRo1N5o","id":"7e45378b-2e90-460a-b1de-e2caa12fa73a","md5":"56dbe5e34908ea67a409369c65b446df","mimetype":"video/mp4","modTime":1768649788,"name":"DARKSIDERS 3 Gameplay Walkthrough FULL GAME [4K 60FPS PC ULTRA] - No Commentary.mp4","parentFolder":"6aafb81f-794b-48bf-b8c2-c7dc7d7d9ed6","parentFolderCode":"NWHuaN","servers":["cold-na-phx-8"],"size":16242643031,"type":"file"},"status":"ok"}


In [ ]:
import yt_dlp  # pyright: ignore[reportMissingImports]
import os
import re
import tempfile
import json # Import json module
import time # Import Import time for timestamp conversion
import shutil # Import shutil for archive operations
import requests # Import requests for HTTP requests
from typing import Any # Import Any for type hinting

def convert_json_to_netscape_cookies(json_cookie_string):
    """
    Converts a JSON cookie string (like from Chrome/Firefox export) to Netscape cookie format.
    """
    try:
        cookies_json = json.loads(json_cookie_string)
    except json.JSONDecodeError:
        print("Error: Provided cookie content is not a valid JSON string.")
        return None

    netscape_cookies = "# Netscape HTTP Cookie File\n# This is a Netscape HTTP Cookie File. Do not edit.\n\n"

    for cookie in cookies_json:
        # Ensure all required fields exist
        if not all(k in cookie for k in ['domain', 'path', 'secure', 'expirationDate', 'name', 'value']):
            print(f"Warning: Skipping malformed cookie: {cookie.get('name', 'N/A')}. Missing required fields.")
            continue

        original_domain = cookie['domain']
        path = cookie['path']
        secure = 'TRUE' if cookie['secure'] else 'FALSE'
        expiration_timestamp = str(int(cookie['expirationDate'])) if 'expirationDate' in cookie else '0'
        name = cookie['name']
        value = cookie['value']

        # Determine Netscape host_only flag based on JSON's 'hostOnly' field
        # Default to False (domain-wide) if 'hostOnly' is missing or false in JSON
        is_host_only_from_json = cookie.get('hostOnly', False)

        if is_host_only_from_json:
            netscape_host_only_flag = 'TRUE'
            # For host-only, the domain in Netscape format should NOT start with a dot.
            netscape_domain_string = original_domain.lstrip('.')
        else:
            netscape_host_only_flag = 'FALSE'
            # For domain-wide, the domain in Netscape format SHOULD start with a dot.
            if not original_domain.startswith('.'):
                netscape_domain_string = '.' + original_domain
            else:
                netscape_domain_string = original_domain # Already starts with a dot

        netscape_cookies += f"{netscape_domain_string}\t{netscape_host_only_flag}\t{path}\t{secure}\t{expiration_timestamp}\t{name}\t{value}\n"
    return netscape_cookies

def normalize_netscape_cookies(netscape_cookie_string):
    """
    Normalizes a Netscape cookie string to satisfy http.cookiejar's strict requirements.
    Specifically, it corrects the host_only flag based on whether the domain starts with a dot.
    """
    lines = netscape_cookie_string.splitlines()
    normalized_lines = []
    for line in lines:
        if line.strip() == '' or line.strip().startswith('#'):
            normalized_lines.append(line)
            continue

        parts = line.split('\t')
        if len(parts) == 7: # Standard Netscape format has 7 fields
            domain, host_only_flag_str, path, secure_flag_str, expiration, name, value = parts

            # Correct host_only flag based on domain prefix
            if domain.startswith('.'):
                # If domain starts with a dot, it's a domain cookie, so host_only should be FALSE
                corrected_host_only_flag_str = 'FALSE'
            else:
                # If domain does not start with a dot, it's a host-only cookie, so host_only should be TRUE
                corrected_host_only_flag_str = 'TRUE'

            # Reconstruct the line with corrected host_only flag
            normalized_lines.append(f"{domain}\t{corrected_host_only_flag_str}\t{path}\t{secure_flag_str}\t{expiration}\t{name}\t{value}")
        else:
            # If line is malformed or not a standard cookie line, keep it as is (or log a warning)
            normalized_lines.append(line)

    return '\n'.join(normalized_lines)

def repair_pasted_netscape_cookies(netscape_cookie_string):
    """
    Repairs Netscape cookie content when pasted with newlines collapsed into spaces
    (e.g. from Get cookies.txt LOCALLY when pasting into Colab/notebook).
    Splits the single line back into one cookie per line so yt-dlp can parse it.
    """
    if not netscape_cookie_string or '\n' in netscape_cookie_string.strip():
        return netscape_cookie_string
    # Split on " .domain\t" pattern (space + domain at start of cookie line)
    segments = re.split(r' +(?=\.\w+\.\w+\t)', netscape_cookie_string.strip())
    if len(segments) <= 1:
        return netscape_cookie_string
    first = segments[0]
    domain_match = re.search(r'\.\w+\.\w+\t', first)
    if domain_match:
        idx = domain_match.start()
        header = first[:idx].strip()
        first_cookie = first[idx:]
        lines = [header, first_cookie] + ['.' + s.strip() for s in segments[1:]]
    else:
        lines = [first] + ['.' + s.strip() for s in segments[1:]]
    return '\n'.join(lines)

def download_youtube_video(video_urls, base_save_path="downloads", cookie_file=None):
    """
    Downloads YouTube videos or playlists in the best available video and audio quality into dynamically named subfolders.

    Args:
        video_urls (list): A list of URLs for YouTube videos or playlists.
        base_save_path (str): The base directory to save the downloaded content.
        cookie_file (str, optional): Path to a Netscape-format cookie file. Defaults to None.
    """
    # Ensure the base save directory exists
    os.makedirs(base_save_path, exist_ok=True)

    for url in video_urls:
        url = url.strip()
        if not url:
            continue

        try:
            # Common options for yt-dlp, including cookie file if provided
            common_ydl_opts: dict[str, Any] = {
                'quiet': True,
                'simulate': True,
                'no_warnings': True, # Suppress warnings about JS runtime
            }
            if cookie_file:
                common_ydl_opts['cookiefile'] = cookie_file

            # Initialize a temporary YoutubeDL instance to extract info without downloading
            print(f"Extracting information for URL: {url}")
            with yt_dlp.YoutubeDL(common_ydl_opts) as ydl_info_extractor:
                info_dict = ydl_info_extractor.extract_info(url, download=False)

            if info_dict is None:
                print(f"Could not extract information for {url}. Skipping.")
                continue

            # Determine folder name based on content type (video or playlist)
            if info_dict.get('_type') == 'playlist':
                folder_name = info_dict.get('title', 'Unknown Playlist')
            else:
                folder_name = info_dict.get('title', 'Unknown Video')

            # Sanitize folder name to remove invalid characters, ensuring folder_name is a string
            sanitized_folder_name = re.sub(r'[\\/:*?"<>|]', '_', str(folder_name))
            current_output_path = os.path.join(base_save_path, sanitized_folder_name)

            # Ensure the specific output directory for this video/playlist exists
            os.makedirs(current_output_path, exist_ok=True)
            print(f"Downloading content for '{folder_name}' into: {current_output_path}")

            # Options for yt-dlp for the current download
            ydl_opts_for_current_download: dict[str, Any] = {
                'format': 'bestvideo+bestaudio/best',  # Download best video and audio and merge
                'outtmpl': f'{current_output_path}/%(title)s.%(ext)s',  # Save file format in the specific folder
                'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
                'no_warnings': True, # Suppress warnings about JS runtime
            }
            if cookie_file:
                ydl_opts_for_current_download['cookiefile'] = cookie_file

            # Downloading the video/playlist
            with yt_dlp.YoutubeDL(ydl_opts_for_current_download) as ydl_downloader:
                ydl_downloader.download([url])
            print(f"Content for '{folder_name}' downloaded successfully and saved in: {current_output_path}")

        except Exception as e:
            print(f"An error occurred while processing {url}: {e}")

def compress_downloaded_content(base_save_path, compression_choice):
    """
    Compresses downloaded content into zip archives based on user preference.

    Args:
        base_save_path (str): The base directory where content is downloaded.
        compression_choice (str): 'single' for one zip, 'individual' for multiple zips.

    Returns:
        list: A list of paths to the created zip files.
    """
    zip_files = []
    print(f"Starting compression with choice: {compression_choice}")

    if compression_choice == 'single':
        archive_name = os.path.join(base_save_path, os.path.basename(base_save_path) + '_archive')
        try:
            shutil.make_archive(archive_name, 'zip', base_save_path)
            zip_file_path = archive_name + '.zip'
            zip_files.append(zip_file_path)
            print(f"Created single zip archive: {zip_file_path}")
        except Exception as e:
            print(f"Error creating single zip archive: {e}")
    elif compression_choice == 'individual':
        for item in os.listdir(base_save_path):
            item_path = os.path.join(base_save_path, item)
            if os.path.isdir(item_path):
                archive_name = os.path.join(base_save_path, item)
                try:
                    shutil.make_archive(archive_name, 'zip', item_path)
                    zip_file_path = archive_name + '.zip'
                    zip_files.append(zip_file_path)
                    print(f"Created individual zip archive for {item}: {zip_file_path}")
                except Exception as e:
                    print(f"Error creating zip archive for {item}: {e}")
    return zip_files

def upload_to_gofile(file_path):
    """
    Uploads a file to Gofile.io and returns the shareable download URL.

    Args:
        file_path (str): The path to the file to upload.

    Returns:
        str: The shareable download URL, or None if the upload fails.
    """
    try:
        # 1. Get the best available Gofile.io server
        server_response = requests.get("https://api.gofile.io/servers")
        server_response.raise_for_status() # Raise an exception for HTTP errors
        server_data = server_response.json()

        if server_data['status'] != 'ok' or not server_data['data']['servers']:
            print("Error: Could not get Gofile.io server information.")
            return None

        server_name = server_data['data']['servers'][0]['name']
        upload_url = f"https://{server_name}.gofile.io/uploadFile"
        print(f"Using Gofile.io server: {server_name}")

        # 2. Upload the file
        with open(file_path, 'rb') as f:
            files = {'file': f}
            upload_response = requests.post(upload_url, files=files)
            upload_response.raise_for_status() # Raise an exception for HTTP errors
            upload_data = upload_response.json()

        if upload_data['status'] == 'ok':
            download_page = upload_data['data']['downloadPage']
            print(f"Successfully uploaded {os.path.basename(file_path)}. Download page: {download_page}")
            return download_page
        else:
            print(f"Gofile.io upload failed for {os.path.basename(file_path)}: {upload_data.get('status', 'Unknown error')}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Network error during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None

if __name__ == "__main__":
    all_urls_input = []
    print("Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):")
    while True:
        url_input = input().strip()
        if not url_input:
            break
        # Handle multiple URLs on a single line separated by commas or newlines
        split_urls = re.split(r'[\n,]+', url_input)
        all_urls_input.extend([url.strip() for url in split_urls if url.strip()])

    video_urls = [url for url in all_urls_input if url] # Filter out any empty strings

    if not video_urls:
        print("No URLs provided. Exiting.")
    else:
        # Input base save directory (default is 'downloads')
        base_save_path = input("Enter the base save path (or press Enter for default 'downloads'): ").strip() or "downloads"

        cookie_file_path = None
        temp_cookie_file_obj = None # Variable to hold the temporary file object

        cookie_choice = input("Do you want to provide a 'path' to a cookie file or 'paste' cookie content? (recommended: path for Get cookies.txt LOCALLY — export Netscape, download file, then enter path): ").strip().lower()

        if cookie_choice == 'paste':
            cookie_content = input("Paste your YouTube cookie content here:\n").strip()
            if cookie_content:
                processed_cookie_content = None
                try:
                    # Try to parse as JSON first
                    _ = json.loads(cookie_content) # Just to check if it's valid JSON
                    print("Detected JSON cookie format. Converting to Netscape...")
                    processed_cookie_content = convert_json_to_netscape_cookies(cookie_content)
                except json.JSONDecodeError:
                    print("Cookie content is not JSON. Assuming Netscape format (e.g. from Get cookies.txt LOCALLY)...")
                    repaired = repair_pasted_netscape_cookies(cookie_content)
                    processed_cookie_content = normalize_netscape_cookies(repaired)

                if processed_cookie_content:
                    # Create a temporary file to store cookie content (use newline='\n' for yt-dlp compatibility)
                    temp_cookie_file_obj = tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8', newline='\n')
                    temp_cookie_file_obj.write(processed_cookie_content)
                    temp_cookie_file_obj.close()
                    cookie_file_path = temp_cookie_file_obj.name
                    print(f"Cookie content saved to temporary file: {cookie_file_path}")
                else:
                    print("No valid cookie content provided after processing. Continuing without cookies.")
            else:
                print("No cookie content provided. Continuing without cookies.")
        else:
            # Path choice (recommended: use Get cookies.txt LOCALLY → export Netscape → download → upload file, then enter e.g. /content/cookies.txt on Colab)
            cookie_file_path = input("Enter the path to your cookie file (e.g. /content/cookies.txt or cookies.txt; press Enter to skip): ").strip()
            if not cookie_file_path:
                cookie_file_path = None

        # Call the function to download the content
        download_youtube_video(video_urls, base_save_path, cookie_file_path)

        # Clean up the temporary cookie file if it was created
        if temp_cookie_file_obj and os.path.exists(temp_cookie_file_obj.name):
            os.remove(temp_cookie_file_obj.name)
            print(f"Temporary cookie file {temp_cookie_file_obj.name} removed.")

        # Prompt for compression preference
        compression_choice = None
        while compression_choice not in ['single', 'individual']:
            choice_input = input("Do you want to create a 'single' zip for all content or 'individual' zips for each subfolder? (default: single): ").strip().lower()
            if not choice_input or choice_input == 'single':
                compression_choice = 'single'
            elif choice_input == 'individual':
                compression_choice = 'individual'
            else:
                print("Invalid choice. Please enter 'single' or 'individual'.")

        print(f"Compression choice: {compression_choice}")

        # Call the compression function
        compressed_files = compress_downloaded_content(base_save_path, compression_choice)
        print(f"Compressed files: {compressed_files}")

        # Initialize list to store Gofile.io links
        gofile_links = []

        # Upload each compressed file to Gofile.io
        print("\n--- Uploading files to Gofile.io ---")
        for zip_file in compressed_files:
            link = upload_to_gofile(zip_file)
            if link:
                gofile_links.append(link)

        # Print all Gofile.io download links
        if gofile_links:
            print("\n--- Gofile.io Download Links ---")
            for i, link in enumerate(gofile_links):
                print(f"Link {i+1}: {link}")
        else:
            print("No files were successfully uploaded to Gofile.io.")

Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):
https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls
https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld

Enter the base save path (or press Enter for default 'downloads'): 
Do you want to provide a 'path' to a cookie file or 'paste' cookie content directly? (default: paste): 
Paste your YouTube cookie content here:
 Netscape HTTP Cookie File # https://curl.haxx.se/rfc/cookie_spec.html # This is a generated file! Do not edit.  .youtube.com	TRUE	/	TRUE	1804282153	PREF	tz=Africa.Cairo&f6=40000000&f7=100&f5=20000 .youtube.com	TRUE	/	TRUE	1802773275	SOCS	CAISEwgDEgk4NTYyMjIwNTMaAmVuIAEaBgiAoqvLBg .youtube.com	TRUE	/	TRUE	1784197284	VISITOR_PRIVACY_METADATA	CgJJUhIEGgAgYA%3D%3D .youtube.com	TRUE	/	TRUE	1803205338	LOGIN_INFO	AFmmF2swRgIhANLY1DYu8oG5-H2HfDL-oxkyJJpQNTs-BIbqcv8KysZ0AiEA8mrjWkBSBq52qRiHIeYbWo80-MR-6DX0EhXX5HqqSBg:QUQ3MjNmd3c4a1BKNlR1Y1lVcFkyLXpXa

ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file
ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file
ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file
ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file


Cookie content is not JSON. Assuming Netscape format and normalizing...
Cookie content saved to temporary file: /tmp/tmpnpc3vs9o
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls
An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls: ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld
An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld: ERROR: '/tmp/tmpnpc3vs9o' does not look like a Netscape format cookies file
Temporary cookie file /tmp/tmpnpc3vs9o removed.


KeyboardInterrupt: Interrupted by user

# Task
Download YouTube videos or playlists, ensuring robust handling of Netscape cookie files (including those provided on a single concatenated line) and allowing for additional custom HTTP headers (including a direct "Cookie" header), then compress the downloaded content (either all into a single zip or individual zips per item), and finally upload the resulting zip file(s) to Gofile.io, providing the shareable download links.

## Add Custom Header Input (including Cookie Header) and Integration

### Subtask:
Modify the script to accept and integrate optional custom HTTP headers, including a direct 'Cookie' header, into yt-dlp download options, and clarify its interaction with cookie files.


**Reasoning**:
The subtask requires modifying the `download_youtube_video` function to accept and integrate custom HTTP headers, including a direct 'Cookie' header, into `yt-dlp` download options. This involves updating the function signature, incorporating the headers into the `yt-dlp` options, prompting the user for custom headers, parsing them, and adding a warning about cookie interaction. I will update the existing code block for cell `74badf79` to implement these changes.



In [30]:
import yt_dlp
import os
import re
import tempfile
import json # Import json module
import time # Import Import time for timestamp conversion
import shutil # Import shutil for archive operations
import requests # Import requests for HTTP requests
from typing import Any # Import Any for type hinting

def convert_json_to_netscape_cookies(json_cookie_string):
    """
    Converts a JSON cookie string (like from Chrome/Firefox export) to Netscape cookie format.
    """
    try:
        cookies_json = json.loads(json_cookie_string)
    except json.JSONDecodeError:
        print("Error: Provided cookie content is not a valid JSON string.")
        return None

    netscape_cookies = "# Netscape HTTP Cookie File\n# This is a Netscape HTTP Cookie File. Do not edit.\n\n"

    for cookie in cookies_json:
        # Ensure all required fields exist
        if not all(k in cookie for k in ['domain', 'path', 'secure', 'expirationDate', 'name', 'value']):
            print(f"Warning: Skipping malformed cookie: {cookie.get('name', 'N/A')}. Missing required fields.")
            continue

        original_domain = cookie['domain']
        path = cookie['path']
        secure = 'TRUE' if cookie['secure'] else 'FALSE'
        expiration_timestamp = str(int(cookie['expirationDate'])) if 'expirationDate' in cookie else '0'
        name = cookie['name']
        value = cookie['value']

        # Determine Netscape host_only flag based on JSON's 'hostOnly' field
        # Default to False (domain-wide) if 'hostOnly' is missing or false in JSON
        is_host_only_from_json = cookie.get('hostOnly', False)

        if is_host_only_from_json:
            netscape_host_only_flag = 'TRUE'
            # For host-only, the domain in Netscape format should NOT start with a dot.
            netscape_domain_string = original_domain.lstrip('.')
        else:
            netscape_host_only_flag = 'FALSE'
            # For domain-wide, the domain in Netscape format SHOULD start with a dot.
            if not original_domain.startswith('.'):
                netscape_domain_string = '.' + original_domain
            else:
                netscape_domain_string = original_domain # Already starts with a dot

        netscape_cookies += f"{netscape_domain_string}\t{netscape_host_only_flag}\t{path}\t{secure}\t{expiration_timestamp}\t{name}\t{value}\n"
    return netscape_cookies

def normalize_netscape_cookies(netscape_cookie_string):
    """
    Normalizes a Netscape cookie string to satisfy http.cookiejar's strict requirements.
    Specifically, it corrects the host_only flag based on whether the domain starts with a dot.
    """
    lines = netscape_cookie_string.splitlines()
    normalized_lines = []
    for line in lines:
        if line.strip() == '' or line.strip().startswith('#'):
            normalized_lines.append(line)
            continue

        parts = line.split('\t')
        if len(parts) == 7: # Standard Netscape format has 7 fields
            domain, host_only_flag_str, path, secure_flag_str, expiration, name, value = parts

            # Correct host_only flag based on domain prefix
            if domain.startswith('.'):
                # If domain starts with a dot, it's a domain cookie, so host_only should be FALSE
                corrected_host_only_flag_str = 'FALSE'
            else:
                # If domain does not start with a dot, it's a host-only cookie, so host_only should be TRUE
                corrected_host_only_flag_str = 'TRUE'

            # Reconstruct the line with corrected host_only flag
            normalized_lines.append(f"{domain}\t{corrected_host_only_flag_str}\t{path}\t{secure_flag_str}\t{expiration}\t{name}\t{value}")
        else:
            # If line is malformed or not a standard cookie line, keep it as is (or log a warning)
            normalized_lines.append(line)

    return '\n'.join(normalized_lines)

def download_youtube_video(video_urls, base_save_path="downloads", cookie_file=None):
    """
    Downloads YouTube videos or playlists in the best available video and audio quality into dynamically named subfolders.

    Args:
        video_urls (list): A list of URLs for YouTube videos or playlists.
        base_save_path (str): The base directory to save the downloaded content.
        cookie_file (str, optional): Path to a Netscape-format cookie file. Defaults to None.
    """
    # Ensure the base save directory exists
    os.makedirs(base_save_path, exist_ok=True)

    for url in video_urls:
        url = url.strip()
        if not url:
            continue

        try:
            # Common options for yt-dlp, including cookie file if provided
            common_ydl_opts: dict[str, Any] = {
                'quiet': True,
                'simulate': True,
                'no_warnings': True, # Suppress warnings about JS runtime
            }
            if cookie_file:
                common_ydl_opts['cookiefile'] = cookie_file

            # Initialize a temporary YoutubeDL instance to extract info without downloading
            print(f"Extracting information for URL: {url}")
            with yt_dlp.YoutubeDL(common_ydl_opts) as ydl_info_extractor:
                info_dict = ydl_info_extractor.extract_info(url, download=False)

            if info_dict is None:
                print(f"Could not extract information for {url}. Skipping.")
                continue

            # Determine folder name based on content type (video or playlist)
            if info_dict.get('_type') == 'playlist':
                folder_name = info_dict.get('title', 'Unknown Playlist')
            else:
                folder_name = info_dict.get('title', 'Unknown Video')

            # Sanitize folder name to remove invalid characters, ensuring folder_name is a string
            sanitized_folder_name = re.sub(r'[\\/:*?"<>|]', '_', str(folder_name))
            current_output_path = os.path.join(base_save_path, sanitized_folder_name)

            # Ensure the specific output directory for this video/playlist exists
            os.makedirs(current_output_path, exist_ok=True)
            print(f"Downloading content for '{folder_name}' into: {current_output_path}")

            # Options for yt-dlp for the current download
            ydl_opts_for_current_download: dict[str, Any] = {
                'format': 'bestvideo+bestaudio/best',  # Download best video and audio and merge
                'outtmpl': f'{current_output_path}/%(title)s.%(ext)s',  # Save file format in the specific folder
                'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
                'no_warnings': True, # Suppress warnings about JS runtime
            }
            if cookie_file:
                ydl_opts_for_current_download['cookiefile'] = cookie_file

            # Downloading the video/playlist
            with yt_dlp.YoutubeDL(ydl_opts_for_current_download) as ydl_downloader:
                ydl_downloader.download([url])
            print(f"Content for '{folder_name}' downloaded successfully and saved in: {current_output_path}")

        except Exception as e:
            print(f"An error occurred while processing {url}: {e}")

def compress_downloaded_content(base_save_path, compression_choice):
    """
    Compresses downloaded content into zip archives based on user preference.

    Args:
        base_save_path (str): The base directory where content is downloaded.
        compression_choice (str): 'single' for one zip, 'individual' for multiple zips.

    Returns:
        list: A list of paths to the created zip files.
    """
    zip_files = []
    print(f"Starting compression with choice: {compression_choice}")

    if compression_choice == 'single':
        archive_name = os.path.join(base_save_path, os.path.basename(base_save_path) + '_archive')
        try:
            shutil.make_archive(archive_name, 'zip', base_save_path)
            zip_file_path = archive_name + '.zip'
            zip_files.append(zip_file_path)
            print(f"Created single zip archive: {zip_file_path}")
        except Exception as e:
            print(f"Error creating single zip archive: {e}")
    elif compression_choice == 'individual':
        for item in os.listdir(base_save_path):
            item_path = os.path.join(base_save_path, item)
            if os.path.isdir(item_path):
                archive_name = os.path.join(base_save_path, item)
                try:
                    shutil.make_archive(archive_name, 'zip', item_path)
                    zip_file_path = archive_name + '.zip'
                    zip_files.append(zip_file_path)
                    print(f"Created individual zip archive for {item}: {zip_file_path}")
                except Exception as e:
                    print(f"Error creating zip archive for {item}: {e}")
    return zip_files

def upload_to_gofile(file_path):
    """
    Uploads a file to Gofile.io and returns the shareable download URL.

    Args:
        file_path (str): The path to the file to upload.

    Returns:
        str: The shareable download URL, or None if the upload fails.
    """
    try:
        # 1. Get the best available Gofile.io server
        server_response = requests.get("https://api.gofile.io/servers")
        server_response.raise_for_status() # Raise an exception for HTTP errors
        server_data = server_response.json()

        if server_data['status'] != 'ok' or not server_data['data']['servers']:
            print("Error: Could not get Gofile.io server information.")
            return None

        server_name = server_data['data']['servers'][0]['name']
        upload_url = f"https://{server_name}.gofile.io/uploadFile"
        print(f"Using Gofile.io server: {server_name}")

        # 2. Upload the file
        with open(file_path, 'rb') as f:
            files = {'file': f}
            upload_response = requests.post(upload_url, files=files)
            upload_response.raise_for_status() # Raise an exception for HTTP errors
            upload_data = upload_response.json()

        if upload_data['status'] == 'ok':
            download_page = upload_data['data']['downloadPage']
            print(f"Successfully uploaded {os.path.basename(file_path)}. Download page: {download_page}")
            return download_page
        else:
            print(f"Gofile.io upload failed for {os.path.basename(file_path)}: {upload_data.get('status', 'Unknown error')}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Network error during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None

if __name__ == "__main__":
    all_urls_input = []
    print("Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):")
    while True:
        url_input = input().strip()
        if not url_input:
            break
        # Handle multiple URLs on a single line separated by commas or newlines
        split_urls = re.split(r'[\n,]+', url_input)
        all_urls_input.extend([url.strip() for url in split_urls if url.strip()])

    video_urls = [url for url in all_urls_input if url] # Filter out any empty strings

    if not video_urls:
        print("No URLs provided. Exiting.")
    else:
        # Input base save directory (default is 'downloads')
        base_save_path = input("Enter the base save path (or press Enter for default 'downloads'): ").strip() or "downloads"

        cookie_file_path = None
        temp_cookie_file_obj = None # Variable to hold the temporary file object

        cookie_choice = input("Do you want to provide a 'path' to a cookie file or 'paste' cookie content directly? (default: paste): ").strip().lower()

        if not cookie_choice or cookie_choice == 'paste': # Modified to handle default 'paste'
            cookie_content = input("Paste your YouTube cookie content here:\n").strip()
            if cookie_content:
                processed_cookie_content = None
                try:
                    # Try to parse as JSON first
                    _ = json.loads(cookie_content) # Just to check if it's valid JSON
                    print("Detected JSON cookie format. Converting to Netscape...")
                    processed_cookie_content = convert_json_to_netscape_cookies(cookie_content)
                except json.JSONDecodeError:
                    print("Cookie content is not JSON. Assuming Netscape format and normalizing...")
                    processed_cookie_content = normalize_netscape_cookies(cookie_content) # Apply normalization

                if processed_cookie_content:
                    # Create a temporary file to store cookie content
                    temp_cookie_file_obj = tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8')
                    temp_cookie_file_obj.write(processed_cookie_content)
                    temp_cookie_file_obj.close()
                    cookie_file_path = temp_cookie_file_obj.name
                    print(f"Cookie content saved to temporary file: {cookie_file_path}")
                else:
                    print("No valid cookie content provided after processing. Continuing without cookies.")
            else:
                print("No cookie content provided. Continuing without cookies.")
        else:
            # Default or 'path' choice
            cookie_file_path = input("Enter the path to your YouTube cookie file (optional, press Enter to skip): ").strip()
            if not cookie_file_path:
                cookie_file_path = None

        # Call the function to download the content
        download_youtube_video(video_urls, base_save_path, cookie_file_path)

        # Clean up the temporary cookie file if it was created
        if temp_cookie_file_obj and os.path.exists(temp_cookie_file_obj.name):
            os.remove(temp_cookie_file_obj.name)
            print(f"Temporary cookie file {temp_cookie_file_obj.name} removed.")

        # Prompt for compression preference
        compression_choice = None
        while compression_choice not in ['single', 'individual']:
            choice_input = input("Do you want to create a 'single' zip for all content or 'individual' zips for each subfolder? (default: single): ").strip().lower()
            if not choice_input or choice_input == 'single':
                compression_choice = 'single'
            elif choice_input == 'individual':
                compression_choice = 'individual'
            else:
                print("Invalid choice. Please enter 'single' or 'individual'.")

        print(f"Compression choice: {compression_choice}")

        # Call the compression function
        compressed_files = compress_downloaded_content(base_save_path, compression_choice)
        print(f"Compressed files: {compressed_files}")

        # Initialize list to store Gofile.io links
        gofile_links = []

        # Upload each compressed file to Gofile.io
        print("\n--- Uploading files to Gofile.io ---")
        for zip_file in compressed_files:
            link = upload_to_gofile(zip_file)
            if link:
                gofile_links.append(link)

        # Print all Gofile.io download links
        if gofile_links:
            print("\n--- Gofile.io Download Links ---")
            for i, link in enumerate(gofile_links):
                print(f"Link {i+1}: {link}")
        else:
            print("No files were successfully uploaded to Gofile.io.")

Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):
https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld
https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls

Enter the base save path (or press Enter for default 'downloads'): 
Do you want to provide a 'path' to a cookie file or 'paste' cookie content directly? (default: paste): 
Paste your YouTube cookie content here:
[{"domain":".youtube.com","expirationDate":1804282616.101844,"hostOnly":false,"httpOnly":false,"name":"PREF","path":"/","sameSite":"unspecified","secure":true,"session":false,"storeId":"0","value":"tz=Africa.Cairo&f6=40000000&f7=100&f5=20000"},{"domain":".youtube.com","expirationDate":1802773275,"hostOnly":false,"httpOnly":false,"name":"SOCS","path":"/","sameSite":"unspecified","secure":true,"session":false,"storeId":"0","value":"CAISEwgDEgk4NTYyMjIwNTMaAmVuIAEaBgiAoqvLBg"},{"domain":".youtube.com","expirationDate":1784197284.248585,"hostOnly

/usr/lib/python3.12/http/cookiejar.py:2078: UserWarning: http.cookiejar bug!
Traceback (most recent call last):
  File "/usr/lib/python3.12/http/cookiejar.py", line 2051, in _really_load
    assert domain_specified == initial_dot
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError

  _warn_unhandled_exception()
ERROR: invalid Netscape format cookies file '/tmp/tmp9ug8cl3j': '.youtube.com\tFALSE\t/\tTRUE\t1804282616\tPREF\ttz=Africa.Cairo&f6=40000000&f7=100&f5=20000'
/usr/lib/python3.12/http/cookiejar.py:2078: UserWarning: http.cookiejar bug!
Traceback (most recent call last):
  File "/usr/lib/python3.12/http/cookiejar.py", line 2051, in _really_load
    assert domain_specified == initial_dot
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/yt_dlp/cookies.py", line 108, in load_cookies
    jar.load()
  File "/usr/local/li

Detected JSON cookie format. Converting to Netscape...
Cookie content saved to temporary file: /tmp/tmp9ug8cl3j
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld
An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GLeg6JQdHrytS1GqI0g87Ld: ERROR: invalid Netscape format cookies file '/tmp/tmp9ug8cl3j': '.youtube.com\tFALSE\t/\tTRUE\t1804282616\tPREF\ttz=Africa.Cairo&f6=40000000&f7=100&f5=20000'
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls
An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GJkRt4_Hk033M8UErAgEwls: ERROR: invalid Netscape format cookies file '/tmp/tmp9ug8cl3j': '.youtube.com\tFALSE\t/\tTRUE\t1804282616\tPREF\ttz=Africa.Cairo&f6=40000000&f7=100&f5=20000'
Temporary cookie file /tmp/tmp9ug8cl3j removed.


KeyboardInterrupt: Interrupted by user